# Training using PPO2

In [ ]:
from gym_kraby.train import train

train(
    exp_name="test",
    env_name="gym_kraby:OneLegBulletEnv-v0",
    n_envs=32,
    gamma=0.90,  # Discount factor
    n_steps=128,  # batchsize = n_steps * n_envs
    ent_coef=0.01,  # Entropy coefficient for the loss calculation
    learning_rate=10e-4,
    lam=0.95,  # Factor for trade-off of bias vs variance for Generalized Advantage Estimator
    nminibatches=64,  # Number of training minibatches per update.
    noptepochs=30,  # Number of epoch when optimizing the surrogate
    cliprange=0.2,  # Clipping parameter, this clipping depends on the reward scaling, -1 desactivate
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

[+] Starting training 1
[+] Starting training 2
[+] Starting training 3
